<a href="https://colab.research.google.com/github/Thoybur-Rohman/HackTheBox/blob/main/Forensics/New%20Task/New_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Digital Forensics Registry Analysis - Step-by-Step Guide

## 1. Challenge Overview
You were given Windows registry hive files from a compromised computer. The goal: Find out how an attacker maintained persistence (kept access) on the system.

## 2. Why Search the Registry?
Attackers often use the Windows registry to set up programs that run automatically when a user logs in or the system starts. The most common place: `HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Run`. We searched here because it’s a well-known persistence location.

## 3. Searching for Suspicious Entries
```bash
reglookup /home/gitduck/Desktop/forensics_new_task/software | grep -i "CurrentVersion\\Run"
```
This finds all programs set to run at startup. We found a suspicious entry launching PowerShell with a command that loads something from the registry.

## 4. Understanding the Suspicious Command
The command referenced another registry value: `HKLM:SOFTWARE\Microsoft\Windows\CurrentVersion Debug`. This means the actual malicious code (payload) is stored elsewhere in the registry.

## 5. Why Search for "Debug"?
```bash
reglookup /home/gitduck/Desktop/forensics_new_task/software | grep -i "Debug"
```
We searched for "Debug" to find the payload the attacker hid. This is a common trick: use one registry key to trigger code, and another to store the code.

## 6. Extracting the Payload
We found a value like: `/Microsoft/Windows/CurrentVersion/Debug,SZ,<long string>`. The `<long string>` is the actual payload, usually encoded to hide its contents.

## 7. Why Decode? Why Base64?
Attackers encode payloads (often in base64 or Unicode) to avoid detection and make analysis harder. PowerShell often uses UTF-16LE encoding, so we need to decode both base64 and convert the text.

## 8. How to Decode
```bash
# Save the string to a file
echo "<long string>" > payload.b64
# Decode and convert to readable text
base64 -d payload.b64 | iconv -f UTF-16LE -t UTF-8 > decoded_payload.txt
```
This reveals the actual PowerShell script the attacker used.

## 9. What Did We Find?
The decoded script disables security features (like AMSI), downloads more malware, and ensures the attacker keeps access. The flag for the challenge is usually hidden in this script.

## 10. Why These Steps Matter
Searching known persistence locations helps you quickly find how attackers stay on a system. Understanding encoding and decoding lets you reveal hidden payloads. These skills are essential for digital forensics, incident response, and malware analysis.

## 11. Key Takeaways for Beginners
- Always check registry Run keys for persistence.
- Look for references to other registry values—attackers often split their payloads.
- Learn to decode base64 and Unicode—most malware uses these tricks.
- Document every step for repeatability and reporting.
